In [ ]:
# Linking to drive
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
# Loading the drive
import os
os.chdir("/content/gdrive/Shareddrives/Low_Budget_MT")

In [ ]:
#! pip install torch==1.8.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
#! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install . --use-feature=in-tree-build

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/gdrive/Shareddrives/Low_Budget_MT/joeynmt
     |████████████████████████████████| 116 kB 6.9 MB/s 
     |████████████████████████████████| 507 kB 46.2 MB/s 
     |████████████████████████████████| 264 kB 51.5 MB/s 
     |████████████████████████████████| 103 kB 67.0 MB/s 
     |████████████████████████████████| 843 kB 67.3 MB/s 
     |████████████████████████████████| 57 kB 6.3 MB/s 
  Created wheel for joeynmt: filename=joeynmt-1.5.1-py3-none-any.whl size=86509 sha256=112874318e30847177de4e7f51889e6359ee44aaafa8ef63ac8f18785bdeed5b
  Stored in directory: /tmp/pip-ephem-wheel-cache-rwcepuy1/wheels/ec/b0/4e/e15e388dd23d01f1c2b4065e1bb36099e5e22abe4f7147f706
  Created wheel for wrapt: filename=wrapt-1.11.1-cp37-cp37m-linux_x86_64.whl size=68434 sha256=2481fb9aa6561ce913655ecdd58283ec0c9d00103600a169ca05301f1b5c03f0
  Stored in directory: /root/.cache/pip/wheels/4e/58/9d/

In [ ]:
!pip install -U torchtext==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 6.6 MB/s 
     |████████████████████████████████| 831.4 MB 2.7 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.


In [ ]:
# Importing needed libraries for preprocessing and visualization
from my_helpers import split_srctgt, generating_BPE, run_model, nrttl_data_load

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Setting language and data file:
import pandas as pd
# 3. COMET
def comet(files, train_file, number):

  data = {}
  l  = []
  for i in files: 
    with open(i, "r") as src_file:
        x = src_file.read().splitlines()
        l.append(len(x))
        data[i] = x
  assert l.count(l[0]) == len(l), 'Check your files'

  df = pd.DataFrame(data)
  df.columns = ['source_sentence','target_sentence','scores']
  df["scores"] = pd.to_numeric(df["scores"])
  df = df.sort_values(by = ['scores'])
  print(df.head(1))

  return df.head(number), df.tail(df.shape[0]-number)

def comet_data_load(datafile, target_language, query, k, iteration):

  path = os.getcwd() 
  data_path = 'data/'+target_language
  train_file = '/content/gdrive/Shareddrives/Low_Budget_MT/data/sw/'+k+'/'+'swtrain.csv'
  fpath = os.path.join(path, data_path,k,query)
  print(path)
  print(data_path)
  print(fpath)

  if iteration== str(2):
    it = str(int(iteration)-1)
    train = pd.read_csv('/content/gdrive/Shareddrives/Low_Budget_MT/data/'+target_language+'/'+k+'/'+target_language+'train.csv')
    print(train.shape)
    t, rem = comet([fpath+'/1/rem.en', fpath+'/1/rem.'+target_language, fpath+'/1/scores'], train_file, 5000)
    print(t.shape, rem.shape)
    train_full = pd.concat([train,t])

    os.chdir(fpath)

    t.to_csv(iteration+'.csv', index=False)
    train_full.to_csv(query+'_train.csv', index=False)
    rem.to_csv(query+'_rem.csv',index=False)

    os.chdir(path)

  else:
    os.chdir(fpath)

    it = str(int(iteration)-1)

    train, rem = comet([fpath+'/'+it+'/rem.en', fpath+'/'+it+'/rem.'+target_language, fpath+'/'+it+'/scores'], train_file, 5000)
    print(train.shape)
    train.to_csv(iteration+'.csv', index=False)
    train.to_csv(query+'_train.csv',index=False, mode='a', header=False)
    rem.to_csv(query+'_rem.csv',index=False)

    os.chdir(path)

# Amharic COMET

## Iteration 5

### AL Strategy and preprocessing

In [ ]:
comet_data_load('amtrain.csv', 'am', 'comet','k1', '2')

/content/gdrive/Shareddrives/Low_Budget_MT
data/am
/content/gdrive/Shareddrives/Low_Budget_MT/data/am/k1/comet
(30000, 2)
                                         source_sentence  \
86229  Have they not sped? have they not divided the ...   

                                         target_sentence    scores  
86229  ምርኮ አግኝተው የለምን? ተካፍለውስ የለምን? ለእያንዳንዱ ሰው ምርኮ አን...  0.001781  
(20000, 3) (69946, 3)


In [ ]:
split_srctgt('/data/am/k1/comet/comet_train.csv','en', 'am', 'comet','k1', '2')

(50000, 3)


In [ ]:
generating_BPE('en', 'am', 'comet', 'k1','2')

In [ ]:
# Swahili to English
run_model('am', 'am','en','transformer_reverse_enam2_comet','comet','k1','2')
! cd joeynmt; python3 -m joeynmt train configs/k1/transformer_reverse_enam2_comet.yaml

2022-09-01 01:20:54,032 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-09-01 01:20:55,809 - INFO - joeynmt.data - Loading training data...
2022-09-01 01:20:56,936 - INFO - joeynmt.data - Building vocabulary...
2022-09-01 01:20:57,288 - INFO - joeynmt.data - Loading dev data...
2022-09-01 01:20:57,306 - INFO - joeynmt.data - Loading test data...
2022-09-01 01:20:57,829 - INFO - joeynmt.data - Data loaded.
2022-09-01 01:20:57,829 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-09-01 01:20:58,032 - INFO - joeynmt.model - Enc-dec model built.
2022-09-01 01:21:02,762 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-09-01 01:21:03,144 - INFO - joeynmt.training - Total params: 12251136
2022-09-01 01:21:07,071 - INFO - joeynmt.helpers -                           cfg.name : transformer_reverse_enam2_comet
2022-09-01 01:21:07,071 - INFO - joeynmt.helpers -                       cfg.data.src : am
2022-09-01 01:21:07,071 - INFO - joeynmt.helpers -   

In [ ]:
# Output our validation accuracy
! cat "joeynmt/models/k1/transformer_reverse_enam2_comet/validations.txt"

Steps: 600	Loss: 163835.37500	PPL: 87.92627	bleu: 0.08834	LR: 0.00100000	*
Steps: 1200	Loss: 148042.42188	PPL: 57.11034	bleu: 0.51629	LR: 0.00100000	*
Steps: 1800	Loss: 136342.87500	PPL: 41.48438	bleu: 0.90704	LR: 0.00100000	*
Steps: 2400	Loss: 128366.52344	PPL: 33.36068	bleu: 2.15222	LR: 0.00100000	*
Steps: 3000	Loss: 122469.15625	PPL: 28.39585	bleu: 2.15608	LR: 0.00100000	*
Steps: 3600	Loss: 118117.14062	PPL: 25.21231	bleu: 2.36876	LR: 0.00100000	*
Steps: 4200	Loss: 114675.67969	PPL: 22.94961	bleu: 2.44841	LR: 0.00100000	*
Steps: 4800	Loss: 112119.41406	PPL: 21.40138	bleu: 3.01699	LR: 0.00100000	*
Steps: 5400	Loss: 109898.84375	PPL: 20.14151	bleu: 3.09368	LR: 0.00100000	*
Steps: 6000	Loss: 108142.49219	PPL: 19.19776	bleu: 3.20387	LR: 0.00100000	*
Steps: 6600	Loss: 106249.65625	PPL: 18.23013	bleu: 4.11057	LR: 0.00100000	*
Steps: 7200	Loss: 104547.78125	PPL: 17.40182	bleu: 4.15520	LR: 0.00100000	*
Steps: 7800	Loss: 103252.93750	PPL: 16.79692	bleu: 4.16935	LR: 0.00100000	*
Steps: 8400	L

In [ ]:
# Reloading configuration file
ckpt_number = 60000
config = run_model('am', 'am','en','transformer_reverse_enam2_comet','comet','k1','2')
reload_config = config.replace(
    f'#load_model: path ', 
    f'load_model: "/content/gdrive/Shareddrives/Low_Budget_MT/joeynmt/models/k1/transformer_reverse_enam2_comet/{ckpt_number}.ckpt"').replace(
        f'model_dir: "models/k1/transformer_reverse_enam2_comet"', f'model_dir: "models/k1/transformer_reverse_enam2_comet_reload"').replace(
        f'epochs: 60', f'epochs: 8')
        
with open("joeynmt/configs/k1/transformer_reverse_enam2_comet_reload.yaml",'w') as f:
    f.write(reload_config)

In [ ]:
# Train continued
!cd joeynmt; python3 -m joeynmt train configs/k1/transformer_reverse_enam2_comet_reload.yaml

2022-09-01 08:43:46,542 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-09-01 08:43:46,578 - INFO - joeynmt.data - Loading training data...
2022-09-01 08:43:47,709 - INFO - joeynmt.data - Building vocabulary...
2022-09-01 08:43:48,105 - INFO - joeynmt.data - Loading dev data...
2022-09-01 08:43:48,125 - INFO - joeynmt.data - Loading test data...
2022-09-01 08:43:48,629 - INFO - joeynmt.data - Data loaded.
2022-09-01 08:43:48,630 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-09-01 08:43:48,848 - INFO - joeynmt.model - Enc-dec model built.
2022-09-01 08:43:51,323 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-09-01 08:43:51,662 - INFO - joeynmt.training - Total params: 12251136
2022-09-01 08:43:55,148 - INFO - joeynmt.training - Loading model from /content/gdrive/Shareddrives/Low_Budget_MT/joeynmt/models/k1/transformer_reverse_enam2_comet/60000.ckpt
2022-09-01 08:43:55,603 - INFO - joeynmt.helpers -                           cfg.name : t